In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn 
import json
import math
import math
import glob
import cv2
import scipy 
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from PIL import Image, ImageOps
from skimage import io
from skimage.color import rgba2rgb, rgb2xyz
from tqdm import tqdm
from dataclasses import dataclass
from math import floor, ceil
import os

In [ ]:
site = '5d27096c03f801723c31e5e0'
floorNo = 'F3'

floor = plt.imread(f"../input/indoor-location-navigation/metadata/{site}/{floorNo}/floor_image.png")
bw = plt.imread(f"../input/indoorlocationnavigationbwdist/metadata/{site}/{floorNo}/geojson_map_bw_filled_x16.png")
dist = plt.imread(f"../input/indoorlocationnavigationbwdist/metadata/{site}/{floorNo}/geojson_map_cv_dist_normalized_x16.png")

### Indoor Location Competition 2.0 GIT

In [ ]:
!cp -r ../input/indoor-location-competition-20-git/* ./
    # GitHub functions
import io_f 
import visualize_f 
import compute_f
from main import calibrate_magnetic_wifi_ibeacon_to_position
from main import extract_magnetic_strength
from main import extract_wifi_rssi, extract_wifi_count
from main import extract_ibeacon_rssi

In [ ]:
TRAINCOLS = ['site','path','timestamp',
        'acce_x', 'acce_y', 'acce_z', 'acce_accuracy',
        'acce_uncali_x', 'acce_uncali_y', 'acce_uncali_z', 'acce_bias_x', 'acce_bias_y', 'acce_bias_z', 'acce_uncali_accuracy',
        'magn_x', 'magn_y', 'magn_z', 'magn_accuracy',
        'magn_uncali_x', 'magn_uncali_y', 'magn_uncali_z', 'magn_bias_x', 'magn_bias_y', 'magn_bias_z', 'magn_uncali_accuracy',
        'gyro_x', 'gyro_y', 'gyro_z', 'gyro_accuracy',
        'gyro_uncali_x', 'gyro_uncali_y', 'gyro_uncali_z', 'gyro_bias_x', 'gyro_bias_y', 'gyro_bias_z', 'gyro_uncali_accuracy',
        'ahrs_x', 'ahrs_y', 'ahrs_z', 'ahrs_accuracy',
        'wifi_ssid', 'wifi_bssid', 'wifi_rssi', 'wifi_freq', 'wifi_ls_ts',
        'uuid', 'majorid', 'minorid', 'txpow', 'beacon_rssi', 'distance', 'macaddr', 'unix_time',
        'way_x', 'way_y', 'floor'
       ]
floors = {'1F':0, '2F':1, '3F':2, '4F':3, '5F':4, '6F':5, '7F':6, '8F':7, '9F':8, 
          'B':-1, 'B1':-1,'B2':-2, 'B3':-3, 'BF':-1, 'BM':-1, 
          'F1':0, 'F2':1, 'F3':2, 'F4':3, 'F5':4, 'F6':5,'F7':6, 'F8':7, 'F9':8, 'F10':9,
          'G':0, 'LG1':-1, 'LG2':-2, 
          "L1":0,"L2":1,"L3":2,"L4":3,"L5":4,"L6":5,"L7":6,"L8":7,"L9":8,"L10":9,'L11':10, 
          'LM':0, 'M':0, 'P1':-1, 'P2':-2,
          "地下一层":-1,"一层":0,"二层":1,"三层":2,"四层":3}

In [ ]:
#sample path
base = '../input/indoor-location-navigation'
path = f'{base}/train/5cd56b5ae2acfd2d33b58546/F1/5cf396145b96c60008d35a63.txt'
#path = '../input/indoor-location-navigation/train/5d27096c03f801723c31e5e0/F2/5db116392aaa8f000651261f.txt'
# Read in 1 random example
sample_file = io_f.read_data_file(path)
print("~~" + path)
print("acce: {}".format(sample_file.acce.shape), "\n" +
      "acacce_uncalice: {}".format(sample_file.acce_uncali.shape), "\n" +
      "ahrs: {}".format(sample_file.ahrs.shape), "\n" +
      "gyro: {}".format(sample_file.gyro.shape), "\n" +
      "gyro_uncali: {}".format(sample_file.gyro_uncali.shape), "\n" +
      "ibeacon: {}".format(sample_file.ibeacon.shape), "\n" +
      "magn: {}".format(sample_file.magn.shape), "\n" +
      "magn_uncali: {}".format(sample_file.magn_uncali.shape), "\n" +
      "waypoint: {}".format(sample_file.waypoint.shape), "\n" +
      "wifi: {}".format(sample_file.wifi.shape))

In [ ]:
sample_waypoint = pd.DataFrame(sample_file.waypoint, columns = ['timestamp', 'way_x', 'way_y'])
sample_waypoint['timestamp'] = sample_waypoint['timestamp'].astype(int)

print("Waypoint Data")
display(sample_waypoint)

sample_acce = pd.DataFrame(sample_file.acce, columns = ['timestamp', 'acce_x', 'acce_y', 'acce_z'])
sample_gyro = pd.DataFrame(sample_file.gyro, columns = ['timestamp', 'gyro_x', 'gyro_y', 'gyro_z'])
sample_magn = pd.DataFrame(sample_file.magn, columns = ['timestamp', 'magn_x', 'magn_y', 'magn_z'])
sample_ahrs = pd.DataFrame(sample_file.ahrs, columns = ['timestamp', 'ahrs_x', 'ahrs_y', 'ahrs_z'])
com_imu_df = sample_acce.copy()

for df in [sample_acce, sample_gyro, sample_magn, sample_ahrs]:
    
    assert len(com_imu_df) == len(com_imu_df.merge(df, on = 'timestamp'))
    com_imu_df = (com_imu_df.merge(df, on = 'timestamp', suffixes = ("", "_uncali")))

com_imu_df['timestamp'] = com_imu_df['timestamp'].astype(int)

print("Phone Sensors Data")

display(com_imu_df.head())

In [ ]:
#sample_waypoint['timestamp'] = sample_waypoint['timestamp'] - sample_waypoint.timestamp.iloc[0]

imu_df = sample_acce.copy()
imu_df['timestamp'] = com_imu_df['timestamp'] - com_imu_df.timestamp.iloc[0]
imu_df['timestamp'] = imu_df['timestamp'].astype(int)

imu_df['acce_rms'] = np.sqrt(com_imu_df.acce_x**2 + com_imu_df.acce_y**2 + com_imu_df.acce_z**2) - 9.8
new_acce = scipy.ndimage.median_filter(imu_df.acce_rms, 10, mode='constant', cval=0.25)
imu_df['acce_fil'] = new_acce

imu_df['mag_rms'] = np.sqrt(com_imu_df.magn_x**2 + com_imu_df.magn_y**2 + com_imu_df.magn_z**2)
new_mag = scipy.ndimage.median_filter(imu_df.acce_rms, 10, mode='constant', cval=0.25)
imu_df['mag_fil'] = new_mag

imu_df['mag_ang'] = np.arctan(com_imu_df['magn_y']/com_imu_df['magn_x'])
new_mag = scipy.ndimage.median_filter(imu_df.mag_ang, 10, mode='constant', cval=0.25)
imu_df['mag_ang'] = new_mag


In [ ]:
sample_freq = 50
window_size = 22
low_acce_mag = 0.6
step_criterion = 1
interval_threshold = 250

acce_max = np.zeros((2,))
acce_min = np.zeros((2,))
acce_binarys = np.zeros((window_size,), dtype=int)
acce_mag_pre = 0
state_flag = 0

In [ ]:
def init_parameters_filter(sample_freq, warmup_data, cut_off_freq=2):
    order = 4
    filter_b, filter_a = signal.butter(order, cut_off_freq / (sample_freq / 2), 'low', False)
    zf = signal.lfilter_zi(filter_b, filter_a)
    _, zf = signal.lfilter(filter_b, filter_a, warmup_data, zi=zf)
    _, filter_zf = signal.lfilter(filter_b, filter_a, warmup_data, zi=zf)

    return filter_b, filter_a, filter_zf

In [ ]:
from scipy.signal import find_peaks
height = 0.75
width = 8
peak_indexes, pi = find_peaks(imu_df.acce_fil, height, width=width)
inv_data_y = imu_df.acce_fil*(-1) # Tried 1/data_y but not better.
valley_indexes, vi = find_peaks(inv_data_y, height, width=width)

In [ ]:
def synch_peak_vall(peak, valley, timestamp):
    done = True
    while done:
        for i in range(len(valley)):
            print(i, len(peak), len(valley), end='\r')
            if ((i == len(peak)-1) or (i == len(valley)-1)):
                done = False
            try:
                time_diff = timestamp[valley[i]] - timestamp[peak[i]]
            except IndexError:
                break
            if time_diff > 600:
                peak = np.delete(peak, i)
                #print('delete peak', i)
                break
            if time_diff < 0:
                valley = np.delete(valley, i)
                #print('delete valley', i)
                break
    if len(valley)>len(peak):
        valley = np.delete(valley, -1)
        
    elif len(peak)>len(valley):
        peak = np.delete(peak, -1)
        
    return peak, valley

In [ ]:
peak_indexes, valley_indexes = synch_peak_vall(peak_indexes, valley_indexes, imu_df.timestamp)

In [ ]:
display(imu_df)
#text_peak = [str(j) for j in range(len(peak_indexes))]
#text_valley = [str(j) for j in range(len(valley_indexes))]
fig = go.Figure()
fig.add_trace(go.Scatter(x=imu_df.timestamp, y=imu_df.acce_rms, mode='lines', name='Acce RMS'))
fig.add_trace(go.Scatter(x=imu_df.timestamp, y=imu_df.acce_fil, mode='lines', name='Acce LP Filtered'))
fig.add_trace(go.Scatter(x=[imu_df.timestamp[j] for j in peak_indexes], y=[imu_df.acce_fil[j] for j in peak_indexes], mode='markers',text=text_peak, textposition="top center",marker=dict(size=8, color='green'),name='Peaks'))
fig.add_trace(go.Scatter(x=[imu_df.timestamp[j] for j in valley_indexes], y=[imu_df.acce_fil[j] for j in valley_indexes], mode='markers',text=text_valley, textposition="top center", marker=dict(size=8, color='red'),name='Valleys'))
#fig.add_trace(go.Scatter(x=imu_df.timestamp, y=imu_df.mag, mode='lines', name='yaw'))
#for wp in sample_waypoint.timestamp:
#    fig.add_vline(x=wp, line_width=3, line_dash="dash", line_color="green")

fig.update_layout(
    xaxis_title="Time (ms)",
    yaxis_title="Acceleration (m/s<sup>2</sup>)",
    #paper_bgcolor='rgba(0,0,0,0)',
) 

fig.show()

In [ ]:
peak_ts = [imu_df.timestamp[j] for j in valley_indexes]
all_spteps_in_wp = []
spteps_in_wp = 0
wp = 1
peaks = 0
for s in peak_ts:
    peaks += 1
    if wp == len(sample_waypoint.timestamp):
        break
    elif s < sample_waypoint.timestamp[wp]:
        spteps_in_wp += 1
    else:
        all_spteps_in_wp.append(spteps_in_wp)
        spteps_in_wp = 1
        wp += 1
    if peaks == len(peak_ts):
        all_spteps_in_wp.append(spteps_in_wp)

In [ ]:
all_spteps_in_wp

In [ ]:
def step_len(peak, valley, acce, mag):
    steps = []
    for i in range(len(peak)):
        K = 0.68-0.37*mag[peak[i]]+0.15*(mag[peak[i]]**2)
        s_len = K*np.sqrt(np.sqrt(acce[peak[i]]-acce[valley[i]]))
        steps.append(s_len)
    return steps

In [ ]:
step_len = step_len(peak_indexes, valley_indexes, imu_df.acce_fil, imu_df.mag_fil)

In [ ]:
example = read_data_file(path)
org_wp = example.waypoint
org_wp = org_wp[:, 1:3]

# Prepare floor_plan coresponding with our example
site = path.split("/")[4]
floorNo = path.split("/")[5]
floor_plan_filename = f'{base}/metadata/{site}/{floorNo}/floor_image.png'

# Prepare width_meter & height_meter
### (taken from the .json file)
json_plan_filename = f'{base}/metadata/{site}/{floorNo}/floor_info.json'
with open(json_plan_filename) as json_file:
    json_data = json.load(json_file)
    
width_meter = json_data["map_info"]["width"]
height_meter = json_data["map_info"]["height"]
# Title
# Finally, let's plot
visualize_trajectory(trajectory = org_wp,
                     floor_plan_filename = floor_plan_filename,
                     width_meter = width_meter,
                     height_meter = height_meter,
                    title="Original Waypoints")


In [ ]:
visualize_trajectory(trajectory = new_way,
                     floor_plan_filename = floor_plan_filename,
                     width_meter = width_meter,
                     height_meter = height_meter,
                    title="Calculated Waypoints")

In [ ]:
#yaw = [sample_ahrs.ahrs_fil[j] for j in valley_indexes]
med_step = np.mean(stride_lengths[:,1])
x = []
y = []
x.append(org_wp[0][0])
y.append(org_wp[0][1])
for i in range(len(org_wp)-1):
    x.append(x[i]+med_step*all_spteps_in_wp[i]*np.cos(headings[i]))
    y.append(y[i]+med_step*all_spteps_in_wp[i]*np.sin(headings[i]))
    
new_way = list(zip(x,y))
new_way = np.array(new_way)
new_way

In [ ]:
med_step

In [ ]:
def angleFromCoordinate(x1, y1, x2, y2):
    brng = -math.atan2((x2-x1),(y2-y1))
    return brng+1.55

In [ ]:
headings = []
for i in range(1,len(org_wp),1):
    angle = angleFromCoordinate(org_wp[i-1][0], org_wp[i-1][1], org_wp[i][0], org_wp[i][1])
    headings.append(angle)

In [ ]:
headings

In [ ]:
np.degrees(1.1763003931601608)

In [ ]:
angle = angleFromCoordinate(org_wp[0][0], org_wp[0][1], org_wp[1][0], org_wp[1][1])

In [ ]:
swp = sample_file.waypoint[0:2,:]

In [ ]:
swp

In [ ]:
acce_datas = sample_file.acce
ahrs_datas = sample_file.ahrs
posi_datas = sample_file.waypoint

step_timestamps, step_indexs, step_acce_max_mins = compute_f.compute_steps(acce_datas)
headings = compute_f.compute_headings(ahrs_datas)
stride_lengths = compute_f.compute_stride_length(step_acce_max_mins)
step_headings = compute_f.compute_step_heading(step_timestamps, headings)
rel_positions = compute_f.compute_rel_positions(stride_lengths, step_headings)
step_positions = compute_f.correct_positions(rel_positions, posi_datas)

In [ ]:
fig = go.Figure()
#fig.add_trace(go.Scatter(x=sample_acce.timestamp, y=sample_acce.acce, mode='lines', name='original'))
fig.add_trace(go.Scatter(x=imu_df.timestamp, y=imu_df.acce_rms, mode='lines', name='acce'))
fig.add_trace(go.Scatter(x=[imu_df.timestamp[j] for j in step_indexs], y=[imu_df.acce_fil[j] for j in peak_indexes], mode='markers', 
                         text=text_peak, textposition="top center",marker=dict(size=8, color='green'),name='Step'))
#fig.add_trace(go.Scatter(x=imu_df.timestamp, y=imu_df.mag, mode='lines', name='yaw'))
for wp in sample_waypoint.timestamp:
    fig.add_vline(x=wp, line_width=3, line_dash="dash", line_color="green")

fig.update_layout(
    xaxis_title="Time (ms)",
    yaxis_title="Acceleration (m/s^2)",
    #paper_bgcolor='rgba(0,0,0,0)',
) 

fig.show()

In [ ]:
rel_positions[:,0]

In [ ]:
new_way = list(zip(step_positions[:,1],step_positions[:,2]))
new_way = np.array(new_way)
new_way

In [ ]:
org_wp

### DATA IMPORT

In [ ]:
base_dir = "../input/indoor-navigation-and-location-wifi-features/wifi_features"
train_dir = "/train/*_train.csv"
test_dir = "/test/*_test.csv"

# Paths for train & test files
train_paths = sorted(glob.glob(base_dir + train_dir))
test_paths = sorted(glob.glob(base_dir + test_dir))

print("Len Train Files: {}".format(len(train_paths)), "\n" +
      "Len Test Files: {}".format(len(test_paths)))

### Prepeare PySpark

In [ ]:
!pip install pyspark

In [ ]:
## Getting PySpark Ready
import pyspark
from pyspark.sql import SparkSession, Row, SQLContext
from pyspark import SparkContext, sql, SparkConf
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import desc, col, split
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import LinearRegression
conf = SparkConf().setAppName("Indoor Location & Navigation")
conf = (conf.setMaster('local[*]')
        .set('spark.executor.memory', '10G')
        .set('spark.driver.memory', '45G')
        .set('spark.driver.maxResultSize', '10G')
        .set("spark.sql.execution.arrow.enabled", "true"))
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc);
spark = sqlContext.sparkSession
## Add shape funcionality to spark df
def spark_shape(self):
    return (self.count(), len(self.columns))
pyspark.sql.dataframe.DataFrame.shape = spark_shape
# Enable Arrow-based columnar data transfers
#spark.conf.set("spark.sql.execution.arrow.enabled", "true")

### Functions

In [ ]:
def split_data(main_train: pd.DataFrame, train_perc: float):
    ap_qty = main_train.shape[1]-4
    print("~ Number of Paths:", main_train.shape[0], "| Number of APs:", ap_qty)
    main_train = main_train.sort_values('path')
    max_floor = main_train['f'].max()
    min_floor = main_train['f'].min()
    print("~ Floors:", min_floor, "to", max_floor)
    
    ## Skip if use all DF to train
    if (train_perc >= 1):
        print("~ No Split")
        return main_train, None, int(-min_floor)\
    
    ##Find Index to Split by Path
    paths_arr = np.asarray(main_train.groupby(['path']).size())
    total_n = len(paths_arr)
    train_n = int(total_n*0.75)
    test_n = int(total_n-train_n)
    print('~ Paths to train/test:',train_n ,'/',test_n)
    train_path = paths_arr[:train_n]
    test_path = paths_arr[-test_n:]
    train_index = 0
    test_index = 0
    for i in range(train_n):
        train_index = train_index + train_path[i]
    for i in range(test_n):
        test_index = test_index + test_path[i] 
    print("~ Train/test rows:", train_index, "/", test_index)
    
    ## Split by Path
    train = main_train.iloc[:train_index]
    test = main_train.iloc[-test_index:]
    
    return train, test, int(-min_floor), int(ap_qty)

In [ ]:
def assemble_features(pdf: pd.DataFrame, floor_mod: int):
    #print("~ Converting Pandas DF to Spark DF...")
    sdf = spark.createDataFrame(pdf)
    
    #print("~ Assembling all features...")
    feature_cols = sdf.columns[0:-4]
    vect_assem = VectorAssembler(inputCols=feature_cols,outputCol="features")
    features_sdf = vect_assem.transform(sdf)
    
    final_sdf = features_sdf.select(["features","x","y","f","path"])
    final_sdf = final_sdf.withColumn("f", final_sdf.f+floor_mod)
    
    #final_sdf.show()
    #print("~ Done.")
    
    return final_sdf

In [ ]:
def train_floors_spark_lr(train_sdf: spark.createDataFrame):
    #print("~ Floor Logistic Regression Training...")
    f_lr = LogisticRegression(labelCol="f", featuresCol="features")
    
    #print("~~ Training Floor...")
    f_lr_model = f_lr.fit(train_sdf)
    
    print("~ Trainings Complite.")
    return f_lr_model

In [ ]:
def train_way_spark_lr(train_sdf: spark.createDataFrame):
    print("~ X Y Linear Regression Training...")
    x_lr = LinearRegression(labelCol="x", featuresCol="features")
    y_lr = LinearRegression(labelCol="y", featuresCol="features")
    
    print("~~ Training Way X...")
    x_lr_model = x_lr.fit(train_sdf)
    x_sum = x_lr_model.summary
    print("~~~ RMSE: %f" % x_sum.rootMeanSquaredError)
    print("~~~ r2: %f" % x_sum.r2)
    
    print("~~ Training Way Y...")
    y_lr_model = y_lr.fit(train_sdf)
    y_sum = y_lr_model.summary
    print("~~~ RMSE: %f" % y_sum.rootMeanSquaredError)
    print("~~~ r2: %f" % y_sum.r2)
    
    print("~~ Trainings Complite.")
    return x_lr_model, y_lr_model

In [ ]:
def train_way_spark_ir(train_sdf: spark.createDataFrame):
    from pyspark.ml.regression import IsotonicRegression
    print("~ X Y Isotonic Regression Training...")
    x = IsotonicRegression(labelCol="x", featuresCol="features")
    y = IsotonicRegression(labelCol="y", featuresCol="features")
    
    print("~~ Training Way X...")
    x_model = x.fit(train_sdf)
    x_sum = model.summary
    print("~~~ Coefficient Standard Errors: " + str(x_sum.coefficientStandardErrors))
    print("~~ Training Way Y...")
    y_model = y.fit(train_sdf)
    y_sum = model.summary
    print("~~~ Coefficient Standard Errors: " + str(y_sum.coefficientStandardErrors))
    
    print("~~ Trainings Complite.")
    return x_model, y_model

In [ ]:
def train_way_knn(train_pdf: pd.DataFrame, ap: int):
    from sklearn.neighbors import KNeighborsRegressor
    
    features = train_pdf.iloc[:,:-4]
    x_label = train_pdf.iloc[:,-4]
    y_label = train_pdf.iloc[:,-3]
    
    print("~ X Y KNN Regression Training...")
    x = KNeighborsRegressor(n_neighbors=ap)
    y = KNeighborsRegressor(n_neighbors=ap)
    
    print("~~ Training Way X...")
    x_model = x.fit(features, x_label)
    x_r2 = x.score(features, x_label)
    print("~~~ r2: %f" % x_r2)
    
    print("~~ Training Way Y...")
    y_model = y.fit(features, y_label)
    y_r2 = y.score(features, y_label)
    print("~~~ r2: %f" % y_r2)
    
    print("~~ Trainings Complite.")
    return x_model, y_model

In [ ]:
def train_way_spark_dtr(train_sdf: spark.createDataFrame):
    print("~ X Y Decision Tree Regression Training...")
    from pyspark.ml import Pipeline
    from pyspark.ml.regression import DecisionTreeRegressor
    from pyspark.ml.feature import VectorIndexer
    
    # Scale features.
    #featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=16).fit(train_sdf)

    # Train a FM model.
    print("~~ Training Way X...")
    x = DecisionTreeRegressor(labelCol="x",featuresCol="features")
    #x_pl = Pipeline(stages=[featureIndexer, x])
    x_model = x.fit(train_sdf)
    
    print("~~ Training Way Y...")
    y = DecisionTreeRegressor(labelCol="y",featuresCol="features")
    #y_pl = Pipeline(stages=[featureIndexer, y])
    y_model = y.fit(train_sdf)
    
    print("~~ Trainings Complite.")
    return x_model, y_model

In [ ]:
def train_way_spark_gbt(train_sdf: spark.createDataFrame):
    print("~ X Y Gradient-Boosted Tree Regression Training...")
    from pyspark.ml import Pipeline
    from pyspark.ml.regression import GBTRegressor
    from pyspark.ml.feature import VectorIndexer
    
    # Scale features.
    #featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=16).fit(train_sdf)

    # Train a FM model.
    print("~~ Training Way X...")
    x = GBTRegressor(labelCol="x",featuresCol="features")
    #x_pl = Pipeline(stages=[featureIndexer, x_gbt])
    x_model = x.fit(train_sdf)
    
    print("~~ Training Way Y...")
    y = GBTRegressor(labelCol="y",featuresCol="features")
    #y_pl = Pipeline(stages=[featureIndexer, y_gbt])
    y_model = y.fit(train_sdf)
    
    print("~~ Trainings Complite.")
    return x_model, y_model

In [ ]:
def validate_spark_models(test_sdf: spark.createDataFrame, x_model, y_model, f_model):
    from pyspark.ml.evaluation import RegressionEvaluator
    
    print("~ Validating Way X...")
    x_val = x_model.transform(test_sdf)
    x_sum = RegressionEvaluator(labelCol="x", predictionCol="prediction", metricName="r2")
    rmse = x_sum.evaluate(x_val)
    print("~~~ r2: %g" % rmse)
    
    print("~ Validating Way Y...")
    y_val = y_model.transform(test_sdf)
    y_sum = RegressionEvaluator(labelCol="y", predictionCol="prediction", metricName="r2")
    rmse = y_sum.evaluate(y_val)
    print("~~~ r2: %g" % rmse)
    
    print("~ Validating Floors...")
    f_val = f_model.transform(test_sdf)
    f_sum = f_model.summary
    print("~~~ Accuracy: %f" % f_sum.accuracy)
    
    print("~ Validation Complite.\n")
    pred_x = x_val.select('prediction').collect()
    pred_y = y_val.select('prediction').collect()
    pred_f = f_val.select('prediction').collect()
    
    return pred_x, pred_y, pred_f

In [ ]:
def validate_spark_wp(test_sdf: pd.DataFrame, x_model, y_model):
    from pyspark.ml.evaluation import RegressionEvaluator
    
    print("~ Validating Way X...")
    x_val = x_model.transform(test_sdf)
    x_sum = RegressionEvaluator(labelCol="x", predictionCol="prediction", metricName="rmse")
    rmse_x = x_sum.evaluate(x_val)
    x_sum = RegressionEvaluator(labelCol="x", predictionCol="prediction", metricName="r2")
    r2_x = x_sum.evaluate(x_val)
    print("~~~ RMSE: %g" % rmse_x)
    print("~~~ R2: %g" % r2_x)
    
    print("~ Validating Way Y...")
    y_val = y_model.transform(test_sdf)
    y_sum = RegressionEvaluator(labelCol="y", predictionCol="prediction", metricName="rmse")
    rmse_y = y_sum.evaluate(y_val)
    y_sum = RegressionEvaluator(labelCol="y", predictionCol="prediction", metricName="r2")
    r2_y = y_sum.evaluate(y_val)
    print("~~~ RMSE: %g" % rmse_y)
    print("~~~ RMSE: %g" % r2_y)
    
    print("~ Validation Complite.\n")
    pred_x = x_val.select('prediction').collect()
    pred_y = y_val.select('prediction').collect()
    
    return pred_x, pred_y, rmse_x, rmse_y

In [ ]:
def validate_wp(test_sdf: spark.createDataFrame, x_model, y_model):
    from sklearn.metrics import mean_squared_error
    from sklearn.metrics import r2_score
    
    features = test_pdf.iloc[:,:-4]
    x_label = test_pdf.iloc[:,-4]
    y_label = test_pdf.iloc[:,-3]
    
    print("~ Validating Way X...")
    pred_x = x_model.predict(features)
    
    rmse_x = mean_squared_error(x_label, pred_x)
    r2_x = r2_score(x_label, pred_x)
    print("~~~ RMSE: %g" % rmse_x)
    print("~~~ R2: %g" % r2_x)
    
    print("~ Validating Way Y...")
    pred_y = y_model.predict(features)
    
    rmse_y = mean_squared_error(y_label, pred_y)
    r2_y = r2_score(y_label, pred_y)
    print("~~~ RMSE: %g" % rmse_y)
    print("~~~ RMSE: %g" % r2_y)
    
    print("~ Validation Complite.\n")
    
    return pred_x, pred_y, r2_x, r2_y

In [ ]:
def validate_spark_floor(test_sdf: spark.createDataFrame, f_model):
    from pyspark.ml.evaluation import MulticlassClassificationEvaluator 
    
    print("~ Model Summary:")
    trainingSummary = f_model.summary
    train_accuracy = trainingSummary.accuracy
    falsePositiveRate = trainingSummary.weightedFalsePositiveRate
    truePositiveRate = trainingSummary.weightedTruePositiveRate
    fMeasure = trainingSummary.weightedFMeasure()
    precision = trainingSummary.weightedPrecision
    recall = trainingSummary.weightedRecall
    print("~~ Accuracy: %s | FPR: %s | TPR: %s | F-measure: %s | Precision: %s | Recall: %s"
      % (train_accuracy, falsePositiveRate, truePositiveRate, fMeasure, precision, recall))
    
    print("~ Validating Floors...")
    f_val = f_model.transform(test_sdf)
    
    # Select example rows to display.
    #f_val.select("prediction", "f", "path").show(5)

    # Select (prediction, true label) and compute test error
    evaluator = MulticlassClassificationEvaluator(
        labelCol="f", predictionCol="prediction", metricName="accuracy")
    test_accuracy = evaluator.evaluate(f_val)
    print("~~ Test Error: %g " % (1.0 - test_accuracy))
    print("~~ Test Accuracy: %g " % test_accuracy)
    
    print("~ Validation Complite.\n")
    pred_f = f_val.select('prediction').collect()
    
    return pred_f, train_accuracy, test_accuracy

In [ ]:
def val_to_pdf(test_pdf: pd.DataFrame, pred_x: list, pred_y: list, pred_f: list, floor_mod: int):
    print("~ Prepearing Predictions Table...")
    pd.set_option('mode.chained_assignment',None)
    test_pdf = test_pdf[["path","x","y","f"]]
    if pred_x is not None:
        test_pdf['pred_x'] = np.array(pred_x)
    if pred_y is not None:
        test_pdf['pred_y'] = np.array(pred_y)
    if pred_f is not None:
        test_pdf['pred_f'] = np.array(pred_f)
        test_pdf['pred_f'] = test_pdf['pred_f'].astype(int) - floor_mod
    
    print("~ Predictions Table Example:")
    display(test_pdf)
    
    return test_pdf

In [ ]:
jkjk === 1

### Floor Training

In [ ]:
train_perc = 0.75
train_accuracy = []
test_accuracy = []
ap_qtys = []
for train_path in train_paths:
    #train_path = train_paths[2]
    print("Train Path:", train_path)
    train = pd.read_csv(train_path, index_col=0)
    print("SPLITTING DATA")
    train_pdf, valid_pdf, floor_mod, ap_qty = split_data(train, train_perc)
    print("FEATURES ASSAMBLE")
    train_sdf = assemble_features(train_pdf, floor_mod)
    valid_sdf = assemble_features(valid_pdf, floor_mod)
    print("TRAINING")
    model_spark_gbt_x, model_spark_gbt_y = train_way_spark_gbt(train_sdf)
    print("VALIDATION")
    pred_x, pred_y, acc1, acc2 = validate_spark_wp(valid_sdf, model_spark_gbt_x, model_spark_gbt_y)
    predictions_pdf = val_to_pdf(valid_pdf, pred_x, pred_y, None, floor_mod)
    train_accuracy.append(acc1)
    test_accuracy.append(acc2)
    ap_qtys.append(ap_qty)
    break

In [ ]:
s_ap_qtys = [str(i) for i in ap_qtys]

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=s_ap_qtys,
    y=train_accuracy,
    text=train_accuracy,
    textposition='auto',
    name='Train Accuracy',
    marker_color='indianred'
))
fig.add_trace(go.Bar(
    x=s_ap_qtys,
    y=test_accuracy,
    text=test_accuracy,
    textposition='auto',
    name='Test Accuracy',
    marker_color='lightsalmon'
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

In [ ]:
plt.style.use('seaborn-deep')

x = train_accuracy
y = test_accuracy
bins = np.arange(start=0, stop=24, step=1)

plt.hist([x, y], bins, label=['train_accuracy', 'test_accuracy'])
plt.legend(loc='upper right')
plt.show()

### Waypoint Training

In [ ]:
train_path = train_paths[3]
train_perc = 0.75
print("Train Path:", train_path)
train = pd.read_csv(train_path, index_col=0)
print("SPLITTING DATA")
train_pdf, test_pdf, floor_mod, ap_qty = split_data(train, train_perc)
print("FEATURES ASSAMBLE")
train_sdf = assemble_features(train_pdf, floor_mod)
test_sdf = assemble_features(test_pdf, floor_mod)

In [ ]:
print("TRAINING")
model_f = train_floors_spark_lr(train_sdf)
print("VALIDATION")
pred_f, acc1, acc2 = validate_spark_floor(test_sdf, model_f)

In [ ]:
print("SKLearn TRAINING")
model_x_knn, model_y_knn = train_way_knn(train_pdf, 80)

In [ ]:
print("VALIDATION")
pred_x, pred_y, x_knn_r2, y_knn_r2 = validate_wp(test_pdf, model_x_knn, model_y_knn)
knn_predictions_pdf = val_to_pdf(test_pdf, pred_x, pred_y, pred_f, floor_mod)

In [ ]:
one_path = knn_predictions_pdf.loc[knn_predictions_pdf['path'] == '5db116392aaa8f000651261f']
#one_path = one_path.groupby(['path','x','y','f'],as_index=False)[["pred_x","pred_y"]].mean()
display(one_path.sort_values('x'))

In [ ]:
knn_predictions_pdf['error'] = np.sqrt(np.power(knn_predictions_pdf.pred_x - knn_predictions_pdf.x, 2) + np.power(knn_predictions_pdf.pred_y - knn_predictions_pdf.y, 2))

In [ ]:
knn_predictions_pdf['error'].median()

In [ ]:
len_arr = np.arange(len(knn_predictions_pdf.error))
#layout = go.Layout( autosize=False, width=600, height=600)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=len_arr,
    y=knn_predictions_pdf.error,
    marker_color='lightsalmon'
))
fig.show()

In [ ]:
print("SPARK TRAINING")
model_x_lr, model_y_lr = train_way_spark_lr(train_sdf)

In [ ]:
print("VALIDATION")
pred_x, pred_y, acc1, acc2 = validate_spark_wp(test_sdf, model_x_lr, model_y_lr)
predictions_pdf = val_to_pdf(test_pdf, pred_x, pred_y, None, floor_mod)

In [ ]:
predictions_pdf

In [ ]:
one_path = knn_predictions_pdf.loc[knn_predictions_pdf['path'] == '5db2a33b5741f4000680a82b']
one_path = one_path.groupby(['path','x','y','f'],as_index=False)[["pred_x","pred_y"]].mean()
display(one_path)
one_path['vel'] = np.sqrt(one_path.x * one_path.y)
one_path['pred_vel'] = np.sqrt(one_path.pred_x * one_path.pred_x + one_path.pred_y * one_path.pred_y)
one_path['error'] = np.sqrt(np.power(one_path.pred_x - one_path.x, 2) + np.power(one_path.pred_y - one_path.y, 2))

In [ ]:
one_path

In [ ]:
len_arr = np.arange(len(one_path.vel))
layout = go.Layout(
    autosize=False,
    width=600,
    height=600)

fig = go.Figure(layout=layout)
fig.add_trace(go.Scatter(x=len_arr, y=one_path.vel, mode='markers', name='Test Waypoints'))
fig.add_trace(go.Scatter(x=len_arr, y=one_path.pred_vel, mode='markers', name='Pred Waypoints'))
fig.show()

fig = go.Figure(layout=layout)
fig.add_trace(go.Bar(
    x=len_arr,
    y=one_path.error,
    marker_color='lightsalmon'
))
fig.show()

In [ ]:
print("Train Acc: ", np.array(train_accuracy).mean(), "Test Acc:", np.array(test_accuracy).mean())

In [ ]:
print("VALIDATION")
pred_x, pred_y, pred_f = validate_spark_models(test_sdf, dtr_model_x, dtr_model_y, model_f)
predictions_pdf = val_to_pdf(test_pdf, pred_x, pred_y, pred_f, floor_mod)

In [ ]:
#../input/indoor-location-navigation/train/5a0546857ecc773753327266/F1/5e15a2e4f4c3420006d521d8.txt
path_train = train.loc[train['path'] == '5db2a33b5741f4000680a82b']
path_test_sdf = assemble_features(path_train, floor_mod)
pred_x, pred_y, pred_f = validate_spark_models(path_test_sdf, fmr_model_x, model_y, model_f)
path_predictions = val_to_pdf(path_train, pred_x, pred_y, pred_f, floor_mod)

In [ ]:
trajectory = one_path[["x","y"]].to_numpy()
trajectory_trained = one_path[["pred_x","pred_y"]].to_numpy()

# Prepare floor_plan coresponding with our example
site = path.split("/")[4]
floorNo = path.split("/")[5]
floor_plan_filename = f'{base}/metadata/{site}/{floorNo}/floor_image.png'

# Prepare width_meter & height_meter
### (taken from the .json file)
json_plan_filename = f'{base}/metadata/{site}/{floorNo}/floor_info.json'
with open(json_plan_filename) as json_file:
    json_data = json.load(json_file)
    
width_meter = json_data["map_info"]["width"]
height_meter = json_data["map_info"]["height"]

# Title
# Finally, let's plot
title = "Original Waypoint"
org_wp = visualize_trajectory(trajectory = trajectory,
                     floor_plan_filename = floor_plan_filename,
                     width_meter = width_meter,
                     height_meter = height_meter,
                     title = title)
title = "Predicted Waypoint"
pre_wp = visualize_trajectory(trajectory = trajectory_trained,
                     floor_plan_filename = floor_plan_filename,
                     width_meter = width_meter,
                     height_meter = height_meter,
                     title = title)
display(org_wp, pre_wp)

In [ ]:
type(org_wp)

In [ ]:
import plotly.io as pio

pio.show(org_wp)

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=2)

fig.add_trace(
    go.Figure(org_wp),
    row=1, col=1
)

fig.add_trace(
    go.Figure(pre_wp),
    row=1, col=2
)

fig.update_layout(height=600, width=800, title_text="Side By Side Subplots")
fig.show()

In [ ]:
trajectory

In [ ]:
trajectory_trained

### Run Wi-Fi Data

In [ ]:
train_perc = 0.75
for train_path, test_path in zip(train_paths, test_paths):
    # --- Read in data ---
    print("Train Path:", train_path)
    train = pd.read_csv(train_path, index_col=0)
    print("SPLITTING DATA")
    train_pdf, test_pdf, floor_mod = split_data(train, train_perc)
    print("TRAINING")
    model_x, model_y, model_f = train_with_spark(train_pdf, floor_mod)
    print("VALIDATION")
    final_predictions = validate_spark_models(test_pdf, model_x, model_y, model_f, floor_mod)
    
    #train_df = train_df.sample(frac=1, random_state=10)
    # Erase last column (which is "site_path_timestamp")
    #test_df = pd.read_csv(test_path, index_col=0)

    #train_size = int(len(train_df) * train_perc)
    
    break

### USEFULL CODE

In [ ]:
# = final_train_data.filter(col("path") == "5dcf884a94e49000061256be")


### X DecisionTreeRegressor

In [ ]:
from pyspark.ml.regression import DecisionTreeRegressor
dtr = DecisionTreeRegressor(labelCol="x", featuresCol="features")

In [ ]:
# Train model.  This also runs the indexers.
x_gbt_model = dtr.fit(train_df)

In [ ]:
# Make predictions.
x_gbt_predictions = x_gbt_model.transform(test_df)
x_gbt_predictions.show()

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="x", predictionCol="prediction", metricName="r2")
rmse = evaluator.evaluate(x_gbt_predictions)
print("Root Mean Squared Error (R2) on test data = %g" % rmse)

In [ ]:
# Wi-Fi Fingerprinting ALL BSSID
wifi_dir = "../input/indoor-navigation-and-location-wifi-features"
train_dir = "/*_train.csv"
test_dir = "/*_test.csv"


# Paths for train & test files
train_paths = sorted(glob.glob(wifi_dir + train_dir))
test_paths = sorted(glob.glob(wifi_dir + test_dir))
sample_subm = pd.read_csv('../input/indoor-location-navigation/sample_submission.csv',
                          index_col=0)

print("Len Train Files: {}".format(len(train_paths)), "\n" +
      "Len Test Files: {}".format(len(test_paths)))